In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [152]:
books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [153]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [154]:
df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [155]:
df.shape

(1193, 5)

In [156]:
df = df_ratings.merge(df_books,on='isbn',how='left')

In [157]:
users_rating_count = df.groupby('user')['rating'].count().reset_index()
book_rating_count = df.groupby('isbn')['rating'].count().reset_index()
more_than_200_users = users_rating_count[users_rating_count['rating']>= 200].user.to_list()
more_than_100_book = book_rating_count[book_rating_count['rating']>= 100].isbn.to_list()
df = df[df.user.isin(more_than_200_users)]
df = df[df.isbn.isin(more_than_100_book)]

In [158]:
df.shape

(49781, 5)

In [159]:
user_item_pivot = pd.pivot_table(df,index='title', columns='user', 
                                  values='rating'
                                 ).fillna(0)
user_item_matrix = csr_matrix(user_item_pivot.values)

In [160]:
%%time

model_knn = NearestNeighbors(metric='cosine',algorithm='brute',n_neighbors=6)

model_knn.fit(user_item_matrix)

Wall time: 1 ms


NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [161]:
def get_recommends(book = ""):
    query_index = user_item_pivot.index.to_list().index(book)
    distances, indices = model_knn.kneighbors(user_item_pivot.iloc[query_index,:].values.reshape(1, -1),n_neighbors=6)
    
    recommended_books = []
    find_name = []
    reccomended = []

    for i in range(len(distances.flatten())):
        if i ==0:
            find_name.append(user_item_pivot.index[query_index])
        else:
            reccomended.append([user_item_pivot.index[indices.flatten()[i]], distances.flatten()[i]])
    recommended_books.append(find_name)
    recommended_books.append(reccomended)
    return recommended_books

In [162]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
    test_pass = True
    recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
    if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
        test_pass = False
    recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
    recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
    for i in range(2): 
        if recommends[1][i][0] not in recommended_books:
            test_pass = False
        if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
            test_pass = False
    if test_pass:
        print("You passed the challenge! 🎉🎉🎉🎉🎉")
    else:
        print("You haven't passed yet. Keep trying!")

test_book_recommendation()

[["Where the Heart Is (Oprah's Book Club (Paperback))"], [['The Lovely Bones: A Novel', 0.7234864], ['I Know This Much Is True', 0.7677075], ['The Surgeon', 0.7699411], ['The Weight of Water', 0.77085835], ["I'll Be Seeing You", 0.8016211]]]
You haven't passed yet. Keep trying!


In [164]:
books = get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")
for i in books:
    print(i)

['The Queen of the Damned (Vampire Chronicles (Paperback))']
[['The Vampire Lestat (Vampire Chronicles, Book II)', 0.51784116], ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.53763384], ['Interview with the Vampire', 0.73450685], ['The Witching Hour (Lives of the Mayfair Witches)', 0.74486566], ['Catch 22', 0.7939835]]
